In [33]:
from requests import get
from pandas import DataFrame, concat, to_datetime, Timedelta, json_normalize
from time import time
import numpy as np
from seaborn import set, distplot
import matplotlib.pyplot as plt
from math import ceil
from plotly.graph_objects import Scatter
from plotly.express import bar, colors
from dotenv import load_dotenv
from datetime import datetime
from time import time
import json, os

In [34]:
fetchSaleData = True
fetchReturnData = True

lastUpdatedInDays = 1

In [35]:
# current time
currentTime = time()
timeToCheckAgainst = datetime.fromtimestamp(currentTime - lastUpdatedInDays*24*60*60)

if not os.path.exists('data files'):
    os.makedirs('data files')

print('Checking for sale data...')
if os.path.exists("./data files/rawSaleData.json"):
    saleDataTime = datetime.fromtimestamp(os.path.getmtime('./data files/rawSaleData.json'))
    print('Found sale data from:', saleDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if saleDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchSaleData = True
    else:
        print('Will be using this locally saved data...')
        fetchSaleData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchSaleData = True

print('Checking for return data...')
if os.path.exists("./data files/rawReturnData.json"):
    returnDataTime = datetime.fromtimestamp(os.path.getmtime('./data files/rawReturnData.json'))
    print('Found return data from:', returnDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if returnDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchReturnData = True
    else:
        print('Will be using this locally saved data...')
        fetchReturnData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchReturnData = True

Checking for sale data...
Found sale data from: Sun Sep 20 04:37:58 2020
Will be using this locally saved data...
Checking for return data...
Found return data from: Sun Sep 20 04:38:37 2020
Will be using this locally saved data...


# Accessing data

In [36]:
load_dotenv()

domain = os.getenv("URL")

if fetchSaleData:
    print('Getting Sale data... Please wait!')
    startTime = time()

    # get sale data
    response = get('https://' + domain + 'sale')
    saleData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Sale data.')
    
    # save locally for later reuse
    with open('./data files/rawSaleData.json', 'w') as fp:
        json.dump(saleData, fp, indent=4)
else:
    # use locally saved data
    with open('./data files/rawSaleData.json', 'r') as fp:
        saleData = json.load(fp)

In [37]:
if fetchReturnData:
    print('Getting Return data... Please wait!')
    startTime = time()

    # get return data
    response = get('https://' + domain + 'return')
    returnData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Return data.')
    
    # save locally for later reuse
    with open('./data files/rawReturnData.json', 'w') as fp:
        json.dump(returnData, fp, indent=4)
else:
    # use locally saved data
    with open('./data files/rawReturnData.json', 'r') as fp:
        returnData = json.load(fp)

# Building dataframes

In [38]:
# build dataframe
saleDF = DataFrame.from_dict(saleData, orient='columns')

# drop columns
saleDF = saleDF.drop(['scanned_at', 'scanned_at_melbourne_date_time'], axis=1)

# rename
saleDF = saleDF.rename(columns={"scanned_at_melbourne_date": "sale_date", "scanned_at_melbourne_time": "sale_time", "id": "sale_id"})

# merging date and time, converting new column to datetime64 datatype
saleDF['date_time_concat'] = saleDF['sale_date'] + ' ' + saleDF['sale_time']
saleDF['date_time'] = to_datetime(saleDF['date_time_concat'], dayfirst=True)
saleDF = saleDF.drop(['sale_date', 'sale_time', 'date_time_concat'], axis=1)

# filter to get records post 22nd Jan
saleDF = saleDF[saleDF['date_time'] >= '2020/01/22 00:00:00']
saleDF = saleDF.sort_values(by='date_time', ascending = True)

# append column 'type' to sale dataframe (sale = 1)
saleDF['current_type'] = 1
saleDF = saleDF.drop('return_id', axis = 1)

# build dataframe
returnDF = DataFrame.from_dict(json_normalize(returnData), orient='columns')

# dropping columns
returnDF = returnDF.drop(['scanned_at', 'bin_id', 'cafe_id', 'dishwasher_id'], axis=1)

# renaming columns
returnDF = returnDF.rename(columns={"scanned_at_melbourne_date_time.date": "return_date", "scanned_at_melbourne_date_time.time": "return_time", "id": "return_id"})

# merging date and time, converting new column to datetime64 datatype
returnDF['date_time_concat'] = returnDF['return_date'] + ' ' + returnDF['return_time']
returnDF['date_time'] = to_datetime(returnDF['date_time_concat'], dayfirst=True)
returnDF = returnDF.drop(['return_date', 'return_time', 'date_time_concat'], axis=1)

# filter to get records post 22nd Jan
returnDF = returnDF[returnDF['date_time'] >= '2020/01/22 00:00:00']
returnDF = returnDF.sort_values(by='date_time', ascending = True)

# append column 'type' to return dataframe (return = 0)
returnDF['current_type'] = 0
returnDF['cafe_id'] = np.nan

df = concat([saleDF, returnDF], ignore_index=True, sort=False)
df = df[['sale_id', 'return_id', 'current_type', 'cup_id', 'cafe_id', 'date_time']]
df = df.sort_values(by='date_time', ascending = True)

In [39]:
df.head()

,sale_id,return_id,current_type,cup_id,cafe_id,date_time
0,4252.0,NaN,1,8574643943,104.0,2020-01-22 07:46:57
1,4253.0,NaN,1,2496593207,104.0,2020-01-22 07:47:50
2,4254.0,NaN,1,1590850598,104.0,2020-01-22 07:48:44
3,4255.0,NaN,1,7342150520,104.0,2020-01-22 07:49:25
4,4256.0,NaN,1,9678996946,104.0,2020-01-22 07:56:56


Categorising algorithm (lost/not lost)

In [40]:
# listing all unique cup ids
cup_ids = df.cup_id.unique()
# creating empty data frames
lost_list = []

# Function to convert string to datetime 
def convert(date_time): 
    format = '%b %d %Y %I:%M%p' # The format 
    datetime_str = datetime.strptime(date_time, format) 
   
    return datetime_str 
   
# Driver code 
date_time = 'Mar 16 2020 11:59PM'
reference_date = convert(date_time)

# iterating thorugh all cup ids
for cup in cup_ids:
    temp_df = df[df.cup_id == cup]
    # finding time gap between last record and 16th of March
    if (reference_date - temp_df.date_time.iloc[len(temp_df)-1])/np.timedelta64(1, 'D') >= 21:
        lost_list.append(temp_df.cup_id.iloc[0])

Creating separate dataframes for each list

In [41]:
lost_df = df[df['cup_id'].isin(lost_list)]
returnDF = lost_df[lost_df['current_type']==0]
timesUsedDF = returnDF.groupby('cup_id').size().reset_index(name='times_used')
total_cups = len(timesUsedDF['cup_id'])
numCupsByTimesUsedDF = timesUsedDF.groupby('times_used').size().reset_index(name='number_of_cups').sort_values(by='times_used', ascending=False)
numCupsByTimesUsedDF.loc[:, 'proportion'] = round(numCupsByTimesUsedDF['number_of_cups'] / total_cups*100)

numCupsByTimesUsedDF

,times_used,number_of_cups,proportion
5,6,3,1.0
4,5,18,7.0
3,4,35,14.0
2,3,55,22.0
1,2,69,27.0
0,1,75,29.0


In [42]:
# Save locally
numCupsByTimesUsedDF.to_csv('./data files/numCupsLost.csv', index = False)

In [43]:
fig = bar(numCupsByTimesUsedDF, 
          x='times_used', 
          y='number_of_cups',
          color='times_used', 
          color_continuous_scale=colors.sequential.Viridis,
          hover_name='proportion',
          title='Cups Usage Chart - Lost Cups',
          labels={
              'number_of_cups' : 'Number of cups',
              'times_used' : 'Times used'
          },
          width=600,
          height=500)
fig.update_traces(marker=dict(line=dict(width=1,
                                        color='Black')),
                  hovertemplate='<b>%{hovertext}%</b>' +
                  '<br><br>Times used: %{marker.color}' +
                  '<br>Number of cups: %{y}')
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(xaxis = dict(tickmode = 'linear'),
                  annotations=[
                          dict(
                              x=0,
                              y=1.125,
                              showarrow=False,
                              text="description",
                              xref="paper",
                              yref="paper",
                              align="left"
                          )]
                  )

fig.update_traces(texttemplate='%{hovertext}%', textposition='outside')

fig.add_trace(Scatter(
    x=[max(numCupsByTimesUsedDF.times_used)],
    y=[max(numCupsByTimesUsedDF.number_of_cups)],
    mode="text",
    text='Total cups: {}'.format(numCupsByTimesUsedDF.number_of_cups.sum()),
    textposition="top left",
    showlegend=False
))

fig.show()

In [44]:
notlost_df = df[~df['cup_id'].isin(lost_list)]
returnDF = notlost_df[notlost_df['current_type']==0]
timesUsedDF = returnDF.groupby('cup_id').size().reset_index(name='times_used')
total_cups = len(timesUsedDF['cup_id'])
numCupsByTimesUsedDF = timesUsedDF.groupby('times_used').size().reset_index(name='number_of_cups').sort_values(by='times_used', ascending=False)
numCupsByTimesUsedDF.loc[:, 'proportion'] = round(numCupsByTimesUsedDF['number_of_cups'] / total_cups*100)

numCupsByTimesUsedDF

,times_used,number_of_cups,proportion
10,11,3,0.0
9,10,9,1.0
8,9,25,2.0
7,8,81,8.0
6,7,116,11.0
5,6,142,13.0
4,5,140,13.0
3,4,126,12.0
2,3,178,16.0
1,2,142,13.0


In [45]:
# Save locally
numCupsByTimesUsedDF.to_csv('./data files/numCupsNotLost.csv', index = False)

In [46]:
fig = bar(numCupsByTimesUsedDF, 
          x='times_used', 
          y='number_of_cups',
          color='times_used', 
          color_continuous_scale=colors.sequential.Viridis,
          hover_name='proportion',
          title='Cups Usage Chart - Not Lost Cups',
          labels={
              'number_of_cups' : 'Number of cups',
              'times_used' : 'Times used'
          },
          width=600,
          height=500)
fig.update_traces(marker=dict(line=dict(width=1,
                                        color='Black')),
                  hovertemplate='<b>%{hovertext}%</b>' +
                  '<br><br>Times used: %{marker.color}' +
                  '<br>Number of cups: %{y}')
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(xaxis = dict(tickmode = 'linear'),
                  annotations=[
                          dict(
                              x=0,
                              y=1.125,
                              showarrow=False,
                              text="description",
                              xref="paper",
                              yref="paper",
                              align="left"
                          )]
                  )

fig.update_traces(texttemplate='%{hovertext}%', textposition='outside')

fig.add_trace(Scatter(
    x=[max(numCupsByTimesUsedDF.times_used)],
    y=[max(numCupsByTimesUsedDF.number_of_cups)],
    mode="text",
    text='Total cups: {}'.format(numCupsByTimesUsedDF.number_of_cups.sum()),
    textposition="top left",
    showlegend=False
))

fig.show()